In [21]:
import matplotlib.pyplot as plt
import numpy as np

from l5kit.data import ChunkedDataset, LocalDataManager
from l5kit.dataset import EgoDataset, AgentDataset
from l5kit.rasterization import build_rasterizer
from l5kit.configs import load_config_data

import os

In [22]:
# opening the zarr_dataset
os.environ["L5KIT_DATA_FOLDER"] = "../../prediction-dataset/"
cfg = load_config_data("./visualisation_config.yaml")
dm = LocalDataManager()
dataset_path = dm.require(cfg["val_data_loader"]["key"])
zarr_dataset = ChunkedDataset(dataset_path)
zarr_dataset.open()

In [23]:
# using EgoDataset interface to extract AV data
rast = build_rasterizer(cfg, dm)
ego_dataset = EgoDataset(cfg, zarr_dataset, rast)
# agent_dataset = AgentDataset(cfg, zarr_dataset, rast)

In [24]:
# checking which frames belong to the scene 0 to know which frames to analyze
print(zarr_dataset.scenes[0])
# for i in range(len(zarr_dataset.scenes)):
#     print(zarr_dataset.scenes[i][0][0], zarr_dataset.scenes[i][0][1])

([  0, 248], 'host-a013', 1572643684617362176, 1572643709617362176)


In [25]:
# scene_frame_indexes = []
# for i in range(len(zarr_dataset.scenes)):
#     scene_frame_indexes.append([zarr_dataset.scenes[i][0][0], zarr_dataset.scenes[i][0][1]])

# print(scene_frame_indexes)

In [40]:
# extracting position information from ego dataset
position = []
# low = scene_frame_indexes[0][0]
# high = scene_frame_indexes[0][1]

# for i in range(0, 248):
#     position.append(ego_dataset[i]["centroid"])

# measurements from Example 9 of https://www.kalmanfilter.net/multiExamples.html
position = [[-393.66, 300.4], 
            [-375.93, 301.78], 
            [-351.04, 295.1], 
            [-328.96, 305.19], 
            [-299.35, 301.06], 
            [-273.36, 302.05], 
            [-245.89, 300], 
            [-222.58, 303.57], 
            [-198.03, 296.33], 
            [-174.17, 297.65], 
            [-146.32, 297.41], 
            [-123.72, 299.61], 
            [-103.47, 299.6], 
            [-78.23, 302.39], 
            [-52.63, 295.04], 
            [-23.34, 300.09], 
            [25.96, 294.72], 
            [49.72, 298.61], 
            [76.94, 294.64], 
            [95.38, 284.88], 
            [119.83, 272.82], 
            [144.01, 264.93], 
            [161.84, 251.46], 
            [180.56, 241.27], 
            [201.42, 222.98],
            [222.62, 203.73],
            [239.4, 184.1],
            [252.51, 166.12],
            [266.26, 138.71],
            [271.75, 119.71],
            [277.4, 100.41],
            [294.12, 79.76],
            [301.23, 50.62],
            [291.8, 32.99],
            [299.89, 2.14]]

In [41]:
xvar = 3
yvar = 3

In [47]:
# setting up matrices for Kalman Filter
# initial state matrix, "unknown" vehicle location so we set pos, vel, acc to 0
x = np.array([[0],
              [0],
              [0],
              [0],
              [0],
              [0]])

# since velocity and acceleration is unkown, a high estimate uncertainty is set, resulting in a high Kalman Gain
P = np.array([[500, 0, 0, 0, 0, 0],
              [0, 500, 0, 0, 0, 0],
              [0, 0, 500, 0, 0, 0],
              [0, 0, 0, 500, 0, 0],
              [0, 0, 0, 0, 500, 0],
              [0, 0, 0, 0, 0, 500]])

# t = measurement period / delta time in seconds
t = 1
acc_std_dev = 0.2

# state transition matrix, kinematic equations for position, velocity, and acceleration
F = np.array([[1, t, 0.5*(t**2), 0, 0,          0], 
              [0, 1,          t, 0, 0,          0], 
              [0, 0,          1, 0, 0,          0], 
              [0, 0,          0, 1, t, 0.5*(t**2)], 
              [0, 0,          0, 0, 1,          t], 
              [0, 0,          0, 0, 0,          1]])

# process noise matrix
Q = np.array([[0.25, 0.5, 0.5,    0,   0,   0], 
              [ 0.5,   1,   1,    0,   0,   0], 
              [ 0.5,   1,   1,    0,   0,   0],
              [   0,   0,   0, 0.25, 0.5, 0.5],
              [   0,   0,   0,  0.5,   1,   1],
              [   0,   0,   0,  0.5,   1,   1]]) * (acc_std_dev**2)

# measurement uncertainty matrix
R = np.array([[xvar, 0], 
              [0, yvar]])

H = np.array([[1, 0, 0, 0, 0, 0], 
              [0, 0, 0, 1, 0, 0]])

I = np.array([[1, 0, 0, 0, 0, 0],
              [0, 1, 0, 0, 0, 0],
              [0, 0, 1, 0, 0, 0],
              [0, 0, 0, 1, 0, 0],
              [0, 0, 0, 0, 1, 0],
              [0, 0, 0, 0, 0, 1]])

In [48]:
# initialization
P = np.matmul(np.matmul(F, P), np.transpose(F)) + Q
# print(P)

In [49]:
def kalmanFilter(measurements, iterations, x, P, R, H, Q, I):
    for i in range(iterations):
        z = np.array([[measurements[i][0]],
                      [measurements[i][1]]])
        
        # Kalman Gain calculation
        # K = (P * H_T) * (H * P * H_T + R)^-1
        K = np.matmul(np.matmul(P, np.transpose(H)), 
                      np.linalg.inv(np.matmul(np.matmul(H, P), np.transpose(H)) + R))
        print(K)

        # current state estimation
        # x = x + K * (z - H * x)
        x = x + np.matmul(K, (z - np.matmul(H, x)))
        print(x)

        # update estimate uncertainty
        # P = (I - K * H) * P * (I - K * H)_T + K * R * K_T
        # IKH = (I - K * H)
        IKH = I - np.matmul(K, H)
        P = np.matmul(np.matmul(IKH, P), np.transpose(IKH)) + np.matmul(np.matmul(K, R), np.transpose(K))
        print(P)

        # predict state and update estimate uncertainty
        # x = F * x
        # P = F * P * F_T + Q
        x = np.matmul(F, x)
        P = np.matmul(np.matmul(F, P), np.transpose(F)) + Q 
        print(x)
        print(P)
    return (x, P)

In [50]:
a, b = kalmanFilter(measurements=position, iterations=1, x=x, P=P, R=R, H=H, Q=Q, I=I)

[[0.99734045 0.        ]
 [0.66490545 0.        ]
 [0.22164697 0.        ]
 [0.         0.99734045]
 [0.         0.66490545]
 [0.         0.22164697]]
[[-392.6130412 ]
 [-261.74668061]
 [ -87.25354669]
 [ 299.60107091]
 [ 199.73759807]
 [  66.58275015]]
[[  2.99202135   1.99471636   0.66494091   0.           0.
    0.        ]
 [  1.99471636 501.34761217 333.80033865   0.           0.
    0.        ]
 [  0.66494091 333.80033865 444.62382426   0.           0.
    0.        ]
 [  0.           0.           0.           2.99202135   1.99471636
    0.66494091]
 [  0.           0.           0.           1.99471636 501.34761217
  333.80033865]
 [  0.           0.           0.           0.66494091 333.80033865
  444.62382426]]
[[-697.98649516]
 [-349.0002273 ]
 [ -87.25354669]
 [ 532.63004406]
 [ 266.32034822]
 [  66.58275015]]
[[ 953.96030186 1227.03968954  556.79719169    0.            0.
     0.        ]
 [1227.03968954 1613.61211372  778.46416291    0.            0.
     0.        ]
 [ 556